In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.426475,False,26,1,38.0
1,6.181762,False,30,1,38.0
2,8.750147,True,35,1,40.0
3,6.812284,False,31,1,40.0
4,7.500126,False,30,1,38.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9993.000000,10000.00000,10000.000000,9890.000000
mean,7.267211,27.26740,1.033900,38.702123
std,1.343418,6.21316,0.202373,2.624828
min,0.562179,12.00000,1.000000,19.000000
25%,6.624891,22.00000,1.000000,38.000000
50%,7.374463,27.00000,1.000000,39.000000
75%,8.068919,32.00000,1.000000,40.000000
max,12.438481,51.00000,5.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5124
False    4876
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
4675,1,23,1,38.0
2971,1,26,1,40.0
4466,0,21,1,39.0
3723,1,23,1,36.0
4541,0,23,1,39.0


In [9]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [11]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [12]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 5ms/step - loss: 2.0315 - mae: 1.0298 - mse: 2.0315 - val_loss: 1.6050 - val_mae: 0.9900 - val_mse: 1.6050
Epoch 2/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3818 - mae: 0.9129 - mse: 1.3818 - val_loss: 1.2492 - val_mae: 0.8534 - val_mse: 1.2492
Epoch 3/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3587 - mae: 0.9059 - mse: 1.3587 - val_loss: 1.6277 - val_mae: 1.0185 - val_mse: 1.6277
Epoch 4/10
209/209 [==============================] - 1s 5ms/step - loss: 1.3299 - mae: 0.8974 - mse: 1.3299 - val_loss: 1.3107 - val_mae: 0.8948 - val_mse: 1.3107
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3006 - mae: 0.8896 - mse: 1.3006 - val_loss: 1.3162 - val_mae: 0.8957 - val_mse: 1.3162
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2855 - mae: 0.8837 - mse: 1.2855 - val_loss: 1.2090 - val_mae: 0.8437 - val_mse: 1.2090
Epoch 7/10
209/2

In [13]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [14]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.2611737
Actual val:  7.936641432

Predicted val:  6.413724
Actual val:  8.375361333379999

Predicted val:  7.379973
Actual val:  7.67649596284

Predicted val:  6.916145
Actual val:  5.6438339071999994

Predicted val:  7.1955166
Actual val:  6.4683627670799995

Predicted val:  8.09125
Actual val:  8.3114272774

Predicted val:  7.4255567
Actual val:  7.87491199864

Predicted val:  7.402601
Actual val:  7.3744626639

Predicted val:  7.6463776
Actual val:  8.24969784404

Predicted val:  7.714068
Actual val:  7.62578964258



In [15]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [16]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [17]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…